In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics.pairwise import sigmoid_kernel, rbf_kernel, polynomial_kernel
from sklearn import neighbors
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


## 1. Load Data

In [2]:
# Load the features
X = np.loadtxt(open("AD_MRI_DATA/1-CN_AD/1.2-MRI_SurfaceArea.csv","rb"),delimiter=",",skiprows=0)
X = np.array(X, dtype=float)
print("X:" + str(X.shape))

# Load the labels
with open('AD_MRI_DATA/1-CN_AD/1-Phenotype.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    c1 = [row[1]for row in reader]
    
del c1[0]
y = np.array(c1, dtype=int)
print("y:" + str(y.shape))


X:(134, 68)
y:(134,)


## 2. Lasso with Classifiers

### 2.1 Lasso

In [4]:
model = linear_model.LassoCV()
#alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)
3
selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 2.114873661318956
X_selected: (134, 38)


### 2.2 Classifiers

In [5]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC( random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    auc_avg = np.sum(auc) / 5
    print("auc: "+str(auc_avg)+"\n")


SVM
[[ 0 46]
 [ 0 88]]
              precision    recall  f1-score   support

           0       0.66      1.00      0.79        88
           1       0.00      0.00      0.00        46

    accuracy                           0.66       134
   macro avg       0.33      0.50      0.40       134
weighted avg       0.43      0.66      0.52       134

auc: 0.9859114015976761

KNN
[[32 14]
 [ 7 81]]
              precision    recall  f1-score   support

           0       0.85      0.92      0.89        88
           1       0.82      0.70      0.75        46

    accuracy                           0.84       134
   macro avg       0.84      0.81      0.82       134
weighted avg       0.84      0.84      0.84       134

auc: 0.8967465504720409

Gaussian Naive Bayes
[[39  7]
 [ 4 84]]
              precision    recall  f1-score   support

           0       0.92      0.95      0.94        88
           1       0.91      0.85      0.88        46

    accuracy                           0.92   

/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[41  5]
 [ 2 86]]
              precision    recall  f1-score   support

           0       0.95      0.98      0.96        88
           1       0.95      0.89      0.92        46

    accuracy                           0.95       134
   macro avg       0.95      0.93      0.94       134
weighted avg       0.95      0.95      0.95       134

auc: 0.9910675381263616



In [7]:
clf = SVC(C=1, random_state=0, probability=True)

y_pred = cross_val_predict(clf, X_selected, y, cv=5)
auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = np.sum(auc) / 5
print("auc: "+str(auc_avg)+"\n")

[[ 0 46]
 [ 0 88]]
              precision    recall  f1-score   support

           0       0.66      1.00      0.79        88
           1       0.00      0.00      0.00        46

    accuracy                           0.66       134
   macro avg       0.33      0.50      0.40       134
weighted avg       0.43      0.66      0.52       134

auc: 0.9859114015976761



/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2.3 Multi-core SVM

In [8]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = np.sum(acc) / 5
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.0 w3:1.0
score:0.97008547008547


In [9]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
acc_avg = np.sum(acc) / 5
print("auc: "+str(auc_avg)+"\n")

[[44  2]
 [ 2 86]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        88
           1       0.96      0.96      0.96        46

    accuracy                           0.97       134
   macro avg       0.97      0.97      0.97       134
weighted avg       0.97      0.97      0.97       134

auc: 0.9859114015976761



## 3. ElasticNet with Classifiers

### 3.1 ElasticNet

In [10]:
model = ElasticNetCV(random_state=0)
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X, y)

print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected = selector.transform(X)
print("X_selected: " + str(X_selected.shape))

alpha: 4.2297473226379125
X_selected: (134, 18)


### 3.2 Classifiers

In [11]:
for i in range(5):
    # SVM
    if i == 0:
        clf = SVC(C=1, random_state=42, probability=True)
        print("SVM")
        
        
    # KNN
    elif i == 1:
        clf = neighbors.KNeighborsClassifier(n_neighbors=2)
        print("KNN")
        
    # Gaussian Naive Bayes
    elif i == 2:
        clf = GaussianNB()
        print("Gaussian Naive Bayes")
        
    # Decision Tree
    elif i == 3:
        clf = tree.DecisionTreeClassifier()
        print("Decision Tree")
        
    # Random Forest
    else:
        clf = RandomForestClassifier()
        print("Random Forest")
    
    # Model Evaluation
    y_pred = cross_val_predict(clf, X_selected, y, cv=5)
    auc = cross_val_score(clf, X_selected, y, cv=5, scoring="roc_auc")
    print(confusion_matrix(y, y_pred,labels=[1,0]))
    print(classification_report(y, y_pred))
    acc_avg = np.sum(acc) / 5
    print("auc: "+str(auc_avg)+"\n")

SVM
[[23 23]
 [ 0 88]]
              precision    recall  f1-score   support

           0       0.79      1.00      0.88        88
           1       1.00      0.50      0.67        46

    accuracy                           0.83       134
   macro avg       0.90      0.75      0.78       134
weighted avg       0.86      0.83      0.81       134

auc: 0.9859114015976761

KNN
[[37  9]
 [ 0 88]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        88
           1       1.00      0.80      0.89        46

    accuracy                           0.93       134
   macro avg       0.95      0.90      0.92       134
weighted avg       0.94      0.93      0.93       134

auc: 0.9859114015976761

Gaussian Naive Bayes
[[38  8]
 [ 5 83]]
              precision    recall  f1-score   support

           0       0.91      0.94      0.93        88
           1       0.88      0.83      0.85        46

    accuracy                           0.90   

### 3.3 Multi-core SVM

In [12]:
sig_mat = sigmoid_kernel(X_selected)
rbf_mat = rbf_kernel(X_selected)
poly_mat = polynomial_kernel(X_selected)

best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        train_kernel = w1 * sig_mat + w2 * rbf_mat + w3 * poly_mat
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, train_kernel, y, cv=5, scoring="accuracy")
        acc_avg = np.sum(acc) / 5
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.0 w2:0.0 w3:1.0
score:0.97008547008547


In [13]:
train_kernel = best_w1 * sig_mat + best_w2 * rbf_mat + best_w3 * poly_mat
y_pred = cross_val_predict(clf, train_kernel, y, cv=5)
auc = cross_val_score(clf, train_kernel, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
acc_avg = np.sum(acc) / 5
print("auc: "+str(auc_avg)+"\n")

[[43  3]
 [ 1 87]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        88
           1       0.98      0.93      0.96        46

    accuracy                           0.97       134
   macro avg       0.97      0.96      0.97       134
weighted avg       0.97      0.97      0.97       134

auc: 0.9859114015976761

